In [39]:
import sys, os
old = sys.stdout
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

def enablePrint():
    sys.stdout = old

In [51]:
import scipy.io as sio
#import statsmodels.tsa.stattools as stm
import numpy as np
from statsmodels.tsa.ar_model import AR

In [52]:
from matplotlib import pyplot
import math
import time
import pickle

In [ ]:
filename = []
directory = '/Users/apple/Desktop/eeglab14_1_2b/ICA_Data/'
for file in os.listdir(directory):
    if file.endswith(".mat"):
        filename.append(file)
filename.sort()
#filename = filename[31::]
print(filename)
#temp_trial = sio.loadmat(file)['data']

In [60]:
for eegdata in filename:
    filepath = directory + eegdata
    mat_contents = sio.loadmat(filepath)
    ica = mat_contents['data']
    hz = 128
    #a 3 second pre-trial baseline removed
    trial = ica.shape[2]
    print("{}: {}".format("Total number of trials is", trial))
    channel = ica.shape[0]
    print("{}: {}".format("Total number of channels in each trial is", channel))
    timepoint = ica.shape[1]
    print("{}: {}".format("Total number of time points in per channel per trial is", timepoint))
    for t in range(trial):
        toSave = []
        hz = 128
        #a 3 second pre-trial baseline removed
        temp_trial = ica[:,:,t]
        temp_trial = temp_trial[:,hz*3:]
        MGCM = np.zeros((channel,channel))
        for k in range(60):
            enablePrint()
            print(k)
            blockPrint()
            current_trial = temp_trial[:, k*hz:(k+1)*hz]
            for i in range(channel):
                for j in range(channel):
                    if i == j:
                        print("{} -> {}:{}".format(k,i,j))
                        MGCM[i,j] = 1
                    blockPrint()
                    x = np.vstack((np.asarray(current_trial[i,:]), np.asarray(current_trial[j,:]))).T
                    maxlag = 3
                    result = grangercausalitytests(x, maxlag, addconst = True, verbose = True)
                    optimal_lag = -1
                    F_test = -1.0
                    for key in result.keys():
                        _F_test_ = result[key][0]['params_ftest'][0]
                        if _F_test_ > F_test:
                            F_test = _F_test_
                            optimal_lag = key
                    if (result[optimal_lag][0]['params_ftest'][1] < 0.03):
                        MGCM[i,j] = math.log(result[optimal_lag][0]['params_ftest'][0])
                    else:
                        MGCM[i,j] = 0
            enablePrint()
            diag = np.max(MGCM)
            for i in range(channel):
                for j in range(channel):
                    if i == j:
                        MGCM[i,j] = 1
                    else:
                        MGCM[i,j] = MGCM[i,j]/diag
            enablePrint()
            pyplot.matshow(MGCM)
            pyplot.show()
            #np.append(result,MGCM)
            #eegdata = os.path.splitext(eegdata)[0]
            toSave.append(MGCM.tolist());
            #imgname = "img/{0}_{1}_{2}.png".format(eegdata, str(t).zfill(2), str(k).zfill(2))
            #pyplot.imsave(imgname, MGCM)
        name = "LSTM_Data/{0}.data".format(str(t).zfill(2))
        np.save(name, np.asarray(toSave))

NameError: name 'OLS' is not defined

In [34]:
contents = np.load("LSTM_Data/00.data.npy")

In [36]:
enablePrint()
print(contents[0])

[[1.         0.27338572 0.         ... 0.         0.27252933 0.        ]
 [0.         1.         0.         ... 0.         0.         0.25025578]
 [0.         0.         1.         ... 0.         0.40386428 0.38924155]
 ...
 [0.         0.         0.         ... 1.         0.         0.24741004]
 [0.         0.         0.32300738 ... 0.         1.         0.28362414]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [38]:
print(contents[1])

[[1.         0.32578946 0.28806359 ... 0.4846877  0.30608935 0.        ]
 [0.         1.         0.         ... 0.3357059  0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.28224078 0.27749811 0.         ... 1.         0.35610624 0.52190271]
 [0.33479073 0.         0.25774204 ... 0.         1.         0.32878237]
 [0.         0.         0.         ... 0.28613715 0.31178654 1.        ]]
